在一个 2 x 3 的板上（board）有 5 块砖瓦，用数字 1~5 来表示, 以及一块空缺用 0 来表示.

一次移动定义为选择 0 与一个相邻的数字（上下左右）进行交换.

最终当板 board 的结果是 [[1,2,3],[4,5,0]] 谜板被解开。

给出一个谜板的初始状态，返回最少可以通过多少次移动解开谜板，如果不能解开谜板，则返回 -1 。

示例：

输入：board = [[1,2,3],[4,0,5]]
输出：1
解释：交换 0 和 5 ，1 步完成

输入：board = [[1,2,3],[5,4,0]]
输出：-1
解释：没有办法完成谜板

输入：board = [[4,1,2],[5,0,3]]
输出：5
解释：
最少完成谜板的最少移动次数是 5 ，
一种移动路径:
尚未移动: [[4,1,2],[5,0,3]]
移动 1 次: [[4,1,2],[0,5,3]]
移动 2 次: [[0,1,2],[4,5,3]]
移动 3 次: [[1,0,2],[4,5,3]]
移动 4 次: [[1,2,0],[4,5,3]]
移动 5 次: [[1,2,3],[4,5,0]]

输入：board = [[3,2,4],[1,5,0]]
输出：14
提示：

board 是一个如上所述的 2 x 3 的数组.
board[i][j] 是一个 [0, 1, 2, 3, 4, 5] 的排列.

In [2]:
l=[1,2,3]
tuple(l)
l

[1, 2, 3]

In [9]:
from collections import deque


def f(board: list):
    start = board[0] + board[1]
    index_neighbor = {0: (1, 3), 1: (0, 2, 4), 2: (1, 5), 3: (0, 4), 4: (1, 3, 5), 5: (2, 4)}
    target = [1, 2, 3, 4, 5, 0]
    visited = set()
    visited.add(tuple(start))
    queue = deque([start])
    res = 0
    while queue:
        sz = len(queue)
        # 将该层进行扩散
        for _ in range(sz):
            cur = queue.popleft()
            if cur == target:
                return res
            zero = cur.index(0)
            for neighbor in index_neighbor[zero]:
                tmp = cur[:]
                tmp[zero], tmp[neighbor] = tmp[neighbor], tmp[zero]
                if tuple(tmp) not in visited:
                    visited.add(tuple(tmp))
                    queue.append(tmp)
        res += 1 # 扩散完 次数+1
    return -1


f([[1, 2, 3], [5, 4, 0]]), f([[4, 1, 2], [5, 0, 3]])


(-1, 5)

In [11]:
l = [[1, 2, 3], [5, 4, 0]]
l[0][1], l[1][1] = l[1][1], l[0][1]
l

[[1, 4, 3], [5, 2, 0]]

In [5]:
from pprint import pprint
import copy

# 直接矩阵dfs 错误 等厉害了再改
def f(board: list):
    row = 2
    col = 3
    target = [[1, 2, 3], [4, 5, 0]]
    res = float('inf')
    cnt = 0
    visited = []

    def dfs(board: list, sr: int, sc: int, r: int, c: int):
        nonlocal res, cnt
        if r < 0 or r > row - 1 or c < 0 or c > col - 1:
            return
        if board == target:
            res = min(res, cnt)
            return
        # if 0<=r<row and 0<=c<=col:
        _board = copy.deepcopy(board)
        _board[sr][sc], _board[r][c] = _board[r][c], _board[sr][sc]
        if _board not in visited:
            # pprint(_board)
            visited.append(_board[:])
            cnt += 1
            dfs(_board, r, c, r - 1, c)
            dfs(_board, r, c, r + 1, c)
            dfs(_board, r, c, r, c - 1)
            dfs(_board, r, c, r, c + 1)
            cnt -= 1

    for r in range(row):
        for c in range(col):
            if board[r][c] == 0:
                dfs(board, r, c, r, c)
                break
    return res


f([[4, 1, 2], [5, 0, 3]])

12

In [5]:
t = ((1, 2), 2, 3)
target = [1, 2, 3, 4, 5, 0]
def different(board):
    return sum(not i == j for i, j in zip(board, target))
different([1,2,3,5,4,0])

2

In [2]:
# https://leetcode-cn.com/problems/sliding-puzzle/solution/python-a-star-by-qubenhao-5tda/
from collections import deque
import heapq


def f(board: list):
    start = board[0] + board[1]
    # n*m puzzle中，m为奇数时,逆序数对奇偶性要和最终奇偶性相同，否则无解
    if sum(1 for i in range(6) for j in range(i + 1, 6) if start[j] and start[i] > start[j]) % 2 == 1:
        return -1
    target = [1, 2, 3, 4, 5, 0]
    index_neighbor = {0: (1, 3), 1: (0, 2, 4), 2: (1, 5), 3: (0, 4), 4: (1, 3, 5), 5: (2, 4)}

    def different(board):
        return sum(not i == j for i, j in zip(board, target))

    pq = []
    priority = different(start)
    heapq.heappush(pq, (priority, 0, start))
    costs = {tuple(start): 0}
    best = set()
    while pq:
        _, step, cur = heapq.heappop(pq)
        if cur == target:
            return step
        best.add(tuple(cur))
        zero = cur.index(0)
        for indexNei in index_neighbor[zero]:
            nei = cur[:]
            nei[zero], nei[indexNei] = nei[indexNei], nei[zero]
            if tuple(nei) not in best:
                if tuple(nei) not in costs or step + 1 < costs[tuple(nei)]:
                    costs[tuple(nei)] = step + 1
                    priority = step + 1 + different(nei)
                    heapq.heappush(pq, (priority, step + 1, nei))
    return -1


f([[1, 2, 3], [5, 4, 0]]), f([[4, 1, 2], [5, 0, 3]])


(-1, 5)